# (1) Настройка связи с mysql

In [1]:
import pymysql.cursors
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="")
cur = conn.cursor()
cur.execute ("""USE orders;""")

# (2) Определение функций

In [3]:
def GetTicker ():
    print ('Сейчас Вам будет предложено ввести данные для заполнения книги заявок')
    correct = 0
    while correct == 0:
        seccode = input ('Введите тикер: ')
        cur.execute("SELECT count(*) FROM security_type WHERE (seccode = '"+ seccode +"');")
        if cur.fetchall() != [(0,)]:
            correct = 1
        else:
            print ('Вы ввели некорректный тикер. Повторите ввод')
    return(seccode)

def GetTime():
    correct = 0
    while correct == 0:
        time = int(input ('Введите момент времени (момент времени должен быть записан в формате HHMMSSQQQ): '))
        if int(time) in range(100000000,235959999):
            correct = 1
        else:
            print ('Вы ввели некорректный момент времени. Повторите ввод')
    return(time)

def DefineTable(seccode):
    cur.execute("SELECT instrument_type FROM security_type WHERE (seccode = '%s');" %(ticker))
    if cur.fetchall() == [('ordinary share',)]:
        table = 'ordshares'
    else:
        cur.execute("SELECT instrument_type FROM security_type WHERE (seccode = '%s');" % (ticker))
        if cur.fetchall() == [('preffered share',)]:
            table = 'prefshares'
        else:
            table = 'bonds'
    return(table)

def ReverseToDatetime(t):
    t = str(t)
    t = time(int(t[0:2]),int(t[2:4]),int(t[4:6]),int(t[6:9])) #выделяем из числа чч:мм:сс,мс
    d = '29.11.2020'
    d = date(int(d[6:10]),int(d[3:5]),int(d[0:2])) #парсим строку с введенной датой на гггг.мм.дд
    dt = datetime.combine(d, t)
    return(dt)

def lob_almost(data): 
    orders = data
    # Построение книги лимитированных заявок (LOB - limited order book)
    # Устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mrk_ind=orders['PRICE']!=0
    #  Таблица выставленных заявок
    plc=orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(orders['ACTION']=='1') & no_mrk_ind]
    # Таблица отозванных заявок
    wdr=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='0') & no_mrk_ind]
    # Таблица исполненных объемов
    trds=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='2') & no_mrk_ind] 
    trds=trds[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
    trds['ORDERNO']=trds.index
    trds.index=range(trds.shape[0])
    # Формируем книгу заявок через мерж всех рассчитанных показателей
    lob_almost=pd.merge(plc,wdr, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost=pd.merge(lob_almost,trds, on='ORDERNO', how='left')
    lob_almost=lob_almost.rename(columns={'VOLUME':'VOLUME_trds'})
    lob_almost=lob_almost.fillna(0)
    lob_almost['TOTAL_VOL']=lob_almost['VOLUME_plc']-lob_almost['VOLUME_wdr']-lob_almost['VOLUME_trds']
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trds','TOTAL_VOL']][(lob_almost['TOTAL_VOL']!=0)]
    return lob_almost

def place(data_from_postgres):
    # Строим сводную таблицу с объемами на постановку, снятие и сделки
    # это почти стакан, но не законченный
    lob = lob_almost(data_from_postgres)
    
    # Вытаскиваем обыкновенные заявки
    lob_ordinar = lob[lob['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob_ordinar.sort_index(axis = 0, inplace = True, ascending = False)
    lob_ordinar = lob_ordinar.fillna(0)
    
    # Вытаскиваем заявки айсберги
    lob_ice = lob[lob['TOTAL_VOL']<0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob_ice.sort_index(axis = 0, inplace = True, ascending = False)
    lob_ice = lob_ice.fillna(0)
    
    # Сводим все в одной таблице
    lob_fin = pd.merge(lob_ordinar, lob_ice, on = 'PRICE', how = 'outer', suffixes=('','_ice'))
    lob_fin.sort_index(axis = 0, inplace = True, ascending = False)
    lob_fin = lob_fin.fillna(0)
    
    plt.barh(lob_fin.index, -lob_fin['B'], color = 'g', height = 0.2, label = 'Buy')
    plt.barh(lob_fin.index, lob_fin['S'], color = 'r', height = 0.2, label = 'Sell')
    plt.barh(lob_fin.index, -lob_fin['B_ice'], color = 'y', height = 0.2, label = 'Buy_Ice')
    plt.barh(lob_fin.index, lob_fin['S_ice'], color = 'b', height = 0.2, label = 'Sell_Ice')
    plt.legend()

def iceberg(data_from_postgres):
    import pandas as pd
    data = data_from_postgres
    #place - датафрейм (ДФ), который содержит наблюдения только по выставленным заявкам (ACTION = 1)
    #в ДФ содержатся данные по номеру заявки и выставленному объему (видимому)
    #iceberg = 1, если обнаружено, что выставленная заявка - айсберг. 0 - иначе
    #time_identified - время вскрытия заявки типа "айсберг". время указано для заявки, вскрывшей айсберг (снятие заявки или сделка) 
    place = data[data['ACTION']=='1']
    place = place[['ORDERNO', 'VOLUME']]
    place['iceberg'] = 0
    place['time_identified'] = 0
    place.index = place['ORDERNO']
    
    #iceberg - ДФ с данными по снятым заявкам и по сделкам (ACTION = 0 или ACTION = 2)
    iceberg = data[data['ACTION']!='1']
    iceberg = iceberg.reset_index()
    iceberg = iceberg[['ORDERNO', 'VOLUME', 'TIME']]
    
    #Идея: 
    #ДФ place содержит только уникальные наблюдения по каждому ORDERNO (ORDERNO - уникальный ключ для ДФ place)
    #для каждой строки ДФ iceberg находит соответсвующее значение ORDERNO в ДФ place
    #если колонка VOLUME в ДФ place больше нуля, но меньше объема снятия/сделки, то код вычитает объем снятой заявки/сделки.
    #при этом в колонке time_identified выставляется время снятия/сделки (так как это действие вскрыло айсберг)
    #в колонке iceberg ставится значение 1, что идентифицирует айсберг

    #если колонка VOLUME в ДФ place больше нуля и объема снятия/сделки, код вычитает объем снятой заявки/сделки.
    #это действие необходимо, чтобы правильно идентифицировать айсберг, если он будет вскрыт в дальнейшем
    #других действий не совершается

    #в остальных случаях (если айсберг не будет вскрыт / уже был вскрыт) никаких действий с таблицей place не совершается
    #P.S. в остальных случаях на самом деле прибавляется ноль к значению VOLUME в ДФ place, но это лишь технический момент
    #прибавление нуля не повлияет на результаты
    
    for i in range(len(iceberg)):

        order_number = iceberg.loc[i, 'ORDERNO']

        if (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] > place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']
            place.loc[order_number, 'time_identified'] = iceberg.loc[i, 'TIME']
            place.loc[order_number, 'iceberg'] = 1

        elif (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] < place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']

        else:
            place.loc[order_number, 'VOLUME'] += 0
            
    #ice_detected - ДФ, содержащий все вскрытые заявки типа "айсберг" за выбранный период времени
    #Примечание: значение VOLUME отражает объем, на котором айсберг был вскрыт. 
    #В ходе дальнейших торгов VOLUME мог и дальше уходить в отрицательную зону
    ice_detected = place[place['iceberg']==1]
    ice_detected.index.names = ['Index']
    
    #количество вскрытых айсбергов за выбранный период времени
    ice_numb = ice_detected['iceberg'].sum()
    
    #добавление времени выставления заявок, содержащих айсберги
    place = data[data['ACTION']=='1']
    place_chd = place[['ORDERNO', 'BUYSELL','PRICE', 'TIME']]
    output = pd.merge(ice_detected, place_chd, on = 'ORDERNO', how = 'left')
    output['delta'] = output['time_identified'] - output['TIME']
    
    return output

def bid_ask(data_from_postgres, time):

    lob_ = lob_almost(data_from_postgres[data_from_postgres['TIME'] <= time])
    # Вытаскиваем обыкновенные заявки
    lob = lob_[lob_['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    
    bid = lob[lob['B'] > 0].index[0]
    ask = lob[lob['S'] > 0].index[-1]
    bid_ask = {'bid': bid, 'ask': ask}
    
    return bid_ask

def place_from_ask_bid(output, price_step):
    """
    Это метод, а не функция, поэтому не возвращает значения
    Значит, нужно просто вызвать метод, дав на вход предыдущий аутпут
    Присваивать значение оператором "=" не надо
    """
    
    output['Bid/Ask'] = 0
    for i in output.index:
        bid = output.loc[i, 'BUYSELL'] == 'B'
        price = output.loc[i, 'PRICE']
        bid_or_ask = bid_ask(data_from_postgres, output.loc[i, 'TIME'] * 1000)
        output.loc[i, 'Bid/Ask'] = bid_or_ask['bid'] - price if bid else price - bid_or_ask['ask']
    output['numb_of_steps'] = output['Bid/Ask'] / price_step
    
def time_to_detection(output):
    
    import matplotlib.pyplot as plt
    
    delta_0 = len(output[output['delta']==0])
    delta_less_10 = len(output[(output['delta']>0)&(output['delta']<=10)])
    delta_10_30 = len(output[(output['delta']>10)&(output['delta']<=30)])
    delta_30_60 = len(output[(output['delta']>30)&(output['delta']<=60)])
    delta_more_60 = len(output[output['delta']>60])
    plt.bar('0', delta_0, color = 'lime')
    plt.bar('0-10', delta_less_10, color = 'g')
    plt.bar('10-30', delta_10_30, color = 'yellow')
    plt.bar('30-60', delta_30_60, color = 'orange')
    plt.bar('>60', delta_more_60, color = 'red')
    plt.xlabel('Время до обнаружения айсберга после его выставления')
    plt.ylabel('Количество заявок')

# (3) Запрос пользовательских данных

In [4]:
# Запрос пользовательских данных
ticker = GetTicker()
table = DefineTable(ticker)
print('Сейчас Вам будет предложено ввести окончание периода анализа')
time_upper = GetTime()
print ('Время начала по умолчанию совпадает с временем начала основной торговой сессии - 10:00:00,000')
time_lower = 100000000
print ('Сейчас Вам будет предложено ввести шаг цены для выбранной бумаги')
price_step = float(input('Введите шаг цены (разделитель - точка): '))

Сейчас Вам будет предложено ввести данные для заполнения книги заявок
Введите тикер: ALRS
Сейчас Вам будет предложено ввести окончание периода анализа
Введите момент времени (момент времени должен быть записан в формате HHMMSSQQQ): 180000000
Время начала по умолчанию совпадает с временем начала основной торговой сессии - 10:00:00,000
Сейчас Вам будет предложено ввести шаг цены для выбранной бумаги
Введите шаг цены (разделитель - точка): 0.01


# (4) Запрос данных из БД и преобразование их в датафрейм

In [5]:
if table == 'ordshares':
    cur.execute("SELECT * from ordshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
elif table == 'prefshares':
    cur.execute("SELECT * from prefshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
rows = cur.fetchall()
data_from_postgres = pd.DataFrame(rows, columns =['NO','SECCODE','BUYSELL','TIME','ORDERNO','ACTION','PRICE','VOLUME','TRADENO','TRADEPRICE']) 
data_from_postgres['TIME'] = data_from_postgres['TIME'].apply(ReverseToDatetime) # перевод времени в таймстамп
data_from_postgres

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
122288,14075136,ALRS,S,175940800,6849776,0,66.23,200,NaN,NaN
122289,14075145,ALRS,S,175940812,6850696,1,66.27,200,NaN,NaN
122290,14076935,ALRS,B,175947977,6851573,1,66.11,700,NaN,NaN
122291,14078327,ALRS,B,175953818,6852259,1,66.15,400,NaN,NaN


# (5) Таблица с айсбергами и характеристиками заявки

In [15]:
output = iceberg(data_from_postgres)
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta
0,86829,-400,1,100142,S,65.64,100142,0
1,107058,-10500,1,100237,S,65.69,100223,14
2,113992,-7500,1,100239,S,65.69,100237,2
3,120434,-7000,1,100821,S,65.69,100253,328
4,147575,-3400,1,100410,B,65.51,100359,11
...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12
164,6734093,-200,1,174831,B,65.93,174831,0
165,6805069,-1300,1,175549,B,66.05,175549,0
166,6805439,-3000,1,175552,B,66.05,175552,0


# (6) Добавление расстояния от Bid/Ask в ценах на момент выставления заявки¶

In [16]:
place_from_ask_bid(output, price_step)
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta,Bid/Ask,numb_of_steps
0,86829,-400,1,100142,S,65.64,100142,0,0.05,5.0
1,107058,-10500,1,100237,S,65.69,100223,14,0.01,1.0
2,113992,-7500,1,100239,S,65.69,100237,2,0.03,3.0
3,120434,-7000,1,100821,S,65.69,100253,328,0.00,0.0
4,147575,-3400,1,100410,B,65.51,100359,11,0.07,7.0
...,...,...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12,0.00,0.0
164,6734093,-200,1,174831,B,65.93,174831,0,-0.01,-1.0
165,6805069,-1300,1,175549,B,66.05,175549,0,-0.04,-4.0
166,6805439,-3000,1,175552,B,66.05,175552,0,0.00,0.0


In [9]:
if table == 'ordshares':
    cur.execute("SELECT * from ordshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
elif table == 'prefshares':
    cur.execute("SELECT * from prefshares WHERE (seccode = '%s') and (ordertime <= '%s');" % (ticker, time_upper))
rows = cur.fetchall()

In [11]:
type(rows)

list

In [17]:
data_from_postgres = pd.DataFrame(rows, columns =['NO','SECCODE','BUYSELL','TIME','ORDERNO','ACTION','PRICE','VOLUME','TRADENO','TRADEPRICE']) 

In [18]:
data_from_postgres

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4207,610549,ALRS,S,100959903,303989,1,65.72,2300,NaN,NaN
4208,610561,ALRS,S,100959926,303989,0,65.72,2300,NaN,NaN
4209,610564,ALRS,S,100959927,303997,1,65.69,2700,NaN,NaN
4210,610565,ALRS,S,100959928,303988,0,65.71,1000,NaN,NaN


In [23]:
data = data_from_postgres
data

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4207,610549,ALRS,S,100959903,303989,1,65.72,2300,NaN,NaN
4208,610561,ALRS,S,100959926,303989,0,65.72,2300,NaN,NaN
4209,610564,ALRS,S,100959927,303997,1,65.69,2700,NaN,NaN
4210,610565,ALRS,S,100959928,303988,0,65.71,1000,NaN,NaN


In [63]:
orders = data
orders

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4207,610549,ALRS,S,100959903,303989,1,65.72,2300,NaN,NaN
4208,610561,ALRS,S,100959926,303989,0,65.72,2300,NaN,NaN
4209,610564,ALRS,S,100959927,303997,1,65.69,2700,NaN,NaN
4210,610565,ALRS,S,100959928,303988,0,65.71,1000,NaN,NaN


In [64]:
# Построение книги лимитированных заявок (LOB - limited order book)
# Устанавливаем маску лимитных заявок заявках (True - лимитная заявка, False - рыночная заявка)
no_mrk_ind = orders['PRICE']!=0
no_mrk_ind

0       True
1       True
2       True
3       True
4       True
        ... 
4207    True
4208    True
4209    True
4210    True
4211    True
Name: PRICE, Length: 4212, dtype: bool

In [65]:
#  Таблица выставленных лимитных заявок
plc = orders[['ORDERNO','BUYSELL','PRICE','VOLUME']][(orders['ACTION']=='1') & no_mrk_ind]
plc

,ORDERNO,BUYSELL,PRICE,VOLUME
0,2,S,67.75,300
1,87,S,66.36,100
2,313,S,66.28,2000
3,406,B,65.00,100
4,486,S,69.63,900
...,...,...,...,...
4203,303965,S,65.69,2700
4206,303988,S,65.71,1000
4207,303989,S,65.72,2300
4209,303997,S,65.69,2700


In [66]:
# Таблица отозванных заявок
wdr=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='0') & no_mrk_ind]
wdr

,ORDERNO,VOLUME
245,9934,800
247,9941,700
249,10006,700
260,10271,2300
261,10272,2300
...,...,...
4202,303960,2500
4204,303959,1000
4205,303965,2700
4208,303989,2300


In [67]:
# Таблица исполненных объемов
trds=orders[['ORDERNO','VOLUME']][(orders['ACTION']=='2') & no_mrk_ind] 
trds=trds[['ORDERNO','VOLUME']].groupby(['ORDERNO']).sum()
trds['ORDERNO']=trds.index
trds.index=range(trds.shape[0])
trds

,VOLUME,ORDERNO
0,500,2587
1,200,2793
2,300,3217
3,100,3374
4,900,3760
...,...,...
484,400,299849
485,100,300022
486,3000,303569
487,300,303572


In [68]:
# Книга (aka стакан, aka LOB)
lob_almost=pd.merge(plc,wdr, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr
0,2,S,67.75,300,NaN
1,87,S,66.36,100,NaN
2,313,S,66.28,2000,NaN
3,406,B,65.00,100,100.0
4,486,S,69.63,900,900.0
...,...,...,...,...,...
2107,303965,S,65.69,2700,2700.0
2108,303988,S,65.71,1000,1000.0
2109,303989,S,65.72,2300,2300.0
2110,303997,S,65.69,2700,NaN


In [81]:
lob_almost=pd.merge(lob_almost,trds, on='ORDERNO', how='left')
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME
0,2,S,67.75,300,0.0,NaN
1,87,S,66.36,100,0.0,NaN
2,313,S,66.28,2000,0.0,NaN
3,406,B,65.00,100,100.0,NaN
4,486,S,69.63,900,900.0,NaN
...,...,...,...,...,...,...
2107,303965,S,65.69,2700,2700.0,NaN
2108,303988,S,65.71,1000,1000.0,NaN
2109,303989,S,65.72,2300,2300.0,NaN
2110,303997,S,65.69,2700,0.0,NaN


In [82]:
lob_almost=lob_almost.rename(columns={'VOLUME':'VOLUME_trds'})
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trds
0,2,S,67.75,300,0.0,NaN
1,87,S,66.36,100,0.0,NaN
2,313,S,66.28,2000,0.0,NaN
3,406,B,65.00,100,100.0,NaN
4,486,S,69.63,900,900.0,NaN
...,...,...,...,...,...,...
2107,303965,S,65.69,2700,2700.0,NaN
2108,303988,S,65.71,1000,1000.0,NaN
2109,303989,S,65.72,2300,2300.0,NaN
2110,303997,S,65.69,2700,0.0,NaN


In [83]:
lob_almost=lob_almost.fillna(0)
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trds
0,2,S,67.75,300,0.0,0.0
1,87,S,66.36,100,0.0,0.0
2,313,S,66.28,2000,0.0,0.0
3,406,B,65.00,100,100.0,0.0
4,486,S,69.63,900,900.0,0.0
...,...,...,...,...,...,...
2107,303965,S,65.69,2700,2700.0,0.0
2108,303988,S,65.71,1000,1000.0,0.0
2109,303989,S,65.72,2300,2300.0,0.0
2110,303997,S,65.69,2700,0.0,0.0


In [91]:
lob_almost['TOTAL_VOL']=lob_almost['VOLUME_plc']-lob_almost['VOLUME_wdr']-lob_almost['VOLUME_trds'] 
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trds,TOTAL_VOL
0,2,S,67.75,300,0.0,0.0,300.0
1,87,S,66.36,100,0.0,0.0,100.0
2,313,S,66.28,2000,0.0,0.0,2000.0
3,406,B,65.00,100,100.0,0.0,0.0
4,486,S,69.63,900,900.0,0.0,0.0
...,...,...,...,...,...,...,...
2107,303965,S,65.69,2700,2700.0,0.0,0.0
2108,303988,S,65.71,1000,1000.0,0.0,0.0
2109,303989,S,65.72,2300,2300.0,0.0,0.0
2110,303997,S,65.69,2700,0.0,0.0,2700.0


In [93]:
lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trds','TOTAL_VOL']][(lob_almost['TOTAL_VOL']!=0)]
lob_almost

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trds,TOTAL_VOL
0,2,S,67.75,300,0.0,0.0,300.0
1,87,S,66.36,100,0.0,0.0,100.0
2,313,S,66.28,2000,0.0,0.0,2000.0
6,631,S,70.00,110000,0.0,0.0,110000.0
7,648,S,66.59,100,0.0,0.0,100.0
...,...,...,...,...,...,...,...
2070,303637,B,65.42,200,0.0,0.0,200.0
2076,303669,S,65.59,500,0.0,0.0,500.0
2080,303703,B,65.51,500,0.0,0.0,500.0
2110,303997,S,65.69,2700,0.0,0.0,2700.0


In [102]:
lob_almost[['ORDERNO','TOTAL_VOL','PRICE']][(lob_almost['TOTAL_VOL']<0)]

,ORDERNO,TOTAL_VOL,PRICE
772,86829,-400.0,65.64
973,107058,-10500.0,65.69
1026,113992,-7500.0,65.69
1076,120434,-7000.0,65.69
1193,147575,-3400.0,65.51


In [95]:
lob = lob_almost
lob

,ORDERNO,BUYSELL,PRICE,VOLUME_plc,VOLUME_wdr,VOLUME_trds,TOTAL_VOL
0,2,S,67.75,300,0.0,0.0,300.0
1,87,S,66.36,100,0.0,0.0,100.0
2,313,S,66.28,2000,0.0,0.0,2000.0
6,631,S,70.00,110000,0.0,0.0,110000.0
7,648,S,66.59,100,0.0,0.0,100.0
...,...,...,...,...,...,...,...
2070,303637,B,65.42,200,0.0,0.0,200.0
2076,303669,S,65.59,500,0.0,0.0,500.0
2080,303703,B,65.51,500,0.0,0.0,500.0
2110,303997,S,65.69,2700,0.0,0.0,2700.0


In [96]:
 # Вытаскиваем обыкновенные заявки
lob_ordinar = lob[lob['TOTAL_VOL']>0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
lob_ordinar

BUYSELL,B,S
PRICE,,
58.73,3400.0,NaN
60.11,100.0,NaN
60.64,1000.0,NaN
61.00,5100.0,NaN
61.10,1200.0,NaN
...,...,...
69.95,NaN,400.0
70.00,NaN,117100.0
70.20,NaN,200.0


In [97]:
lob_ordinar.sort_index(axis = 0, inplace = True, ascending = False)
lob_ordinar

BUYSELL,B,S
PRICE,,
71.00,NaN,400.0
70.49,NaN,3000.0
70.20,NaN,200.0
70.00,NaN,117100.0
69.95,NaN,400.0
...,...,...
61.10,1200.0,NaN
61.00,5100.0,NaN
60.64,1000.0,NaN


In [98]:
lob_ordinar = lob_ordinar.fillna(0)
lob_ordinar

BUYSELL,B,S
PRICE,,
71.00,0.0,400.0
70.49,0.0,3000.0
70.20,0.0,200.0
70.00,0.0,117100.0
69.95,0.0,400.0
...,...,...
61.10,1200.0,0.0
61.00,5100.0,0.0
60.64,1000.0,0.0


In [99]:
# Вытаскиваем заявки айсберги
lob_ice = lob[lob['TOTAL_VOL']<0].pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
lob_ice

BUYSELL,B,S
PRICE,,
65.51,-3400.0,NaN
65.64,NaN,-400.0
65.69,NaN,-25000.0


In [100]:
lob_ice.sort_index(axis = 0, inplace = True, ascending = False)
lob_ice

BUYSELL,B,S
PRICE,,
65.69,NaN,-25000.0
65.64,NaN,-400.0
65.51,-3400.0,NaN


In [101]:
lob_ice = lob_ice.fillna(0)
lob_ice

BUYSELL,B,S
PRICE,,
65.69,0.0,-25000.0
65.64,0.0,-400.0
65.51,-3400.0,0.0


In [103]:
 # Сводим все в одной таблице
lob_fin = pd.merge(lob_ordinar, lob_ice, on = 'PRICE', how = 'outer', suffixes=('','_ice'))
lob_fin.sort_index(axis = 0, inplace = True, ascending = False)
lob_fin = lob_fin.fillna(0)

In [104]:
lob_fin

BUYSELL,B,S,B_ice,S_ice
PRICE,,,,
71.00,0.0,400.0,0.0,0.0
70.49,0.0,3000.0,0.0,0.0
70.20,0.0,200.0,0.0,0.0
70.00,0.0,117100.0,0.0,0.0
69.95,0.0,400.0,0.0,0.0
...,...,...,...,...
61.10,1200.0,0.0,0.0,0.0
61.00,5100.0,0.0,0.0,0.0
60.64,1000.0,0.0,0.0,0.0


In [106]:
data = data_from_postgres
data
#place - датафрейм (ДФ), который содержит наблюдения только по выставленным заявкам (ACTION = 1)
#в ДФ содержатся данные по номеру заявки и выставленному объему (видимому)
#iceberg = 1, если обнаружено, что выставленная заявка - айсберг. 0 - иначе
#time_identified - время вскрытия заявки типа "айсберг". время указано для заявки, вскрывшей айсберг (снятие заявки или сделка) 

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4207,610549,ALRS,S,100959903,303989,1,65.72,2300,NaN,NaN
4208,610561,ALRS,S,100959926,303989,0,65.72,2300,NaN,NaN
4209,610564,ALRS,S,100959927,303997,1,65.69,2700,NaN,NaN
4210,610565,ALRS,S,100959928,303988,0,65.71,1000,NaN,NaN


In [108]:
place = data[data['ACTION']=='1']
place

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4203,610500,ALRS,S,100959792,303965,1,65.69,2700,NaN,NaN
4206,610548,ALRS,S,100959901,303988,1,65.71,1000,NaN,NaN
4207,610549,ALRS,S,100959903,303989,1,65.72,2300,NaN,NaN
4209,610564,ALRS,S,100959927,303997,1,65.69,2700,NaN,NaN


In [111]:
place = place[['ORDERNO', 'VOLUME']]
place['iceberg'] = 0
place['time_identified'] = 0
place

,ORDERNO,VOLUME,iceberg,time_identified
0,2,300,0,0
1,87,100,0,0
2,313,2000,0,0
3,406,100,0,0
4,486,900,0,0
...,...,...,...,...
4203,303965,2700,0,0
4206,303988,1000,0,0
4207,303989,2300,0,0
4209,303997,2700,0,0


In [112]:
place.index = place['ORDERNO']
place

,ORDERNO,VOLUME,iceberg,time_identified
ORDERNO,,,,
2,2,300,0,0
87,87,100,0,0
313,313,2000,0,0
406,406,100,0,0
486,486,900,0,0
...,...,...,...,...
303965,303965,2700,0,0
303988,303988,1000,0,0
303989,303989,2300,0,0


In [117]:
iceberg = data[data['ACTION']!='1']
iceberg = iceberg.reset_index()
iceberg = iceberg[['ORDERNO', 'VOLUME', 'TIME']]
iceberg

,ORDERNO,VOLUME,TIME
0,9367,300,100000117
1,5591,300,100000117
2,9405,900,100000322
3,5591,900,100000322
4,9424,2000,100000343
...,...,...,...
2059,303960,2500,100959790
2060,303959,1000,100959793
2061,303965,2700,100959901
2062,303989,2300,100959926


In [118]:
for i in range(len(iceberg)):

        order_number = iceberg.loc[i, 'ORDERNO']

        if (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] > place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']
            place.loc[order_number, 'time_identified'] = iceberg.loc[i, 'TIME']
            place.loc[order_number, 'iceberg'] = 1

        elif (place.loc[order_number, 'VOLUME'] >= 0) and (iceberg.loc[i, 'VOLUME'] < place.loc[order_number, 'VOLUME']):
            place.loc[order_number, 'VOLUME'] -= iceberg.loc[i, 'VOLUME']

        else:
            place.loc[order_number, 'VOLUME'] += 0

In [122]:
test = place[place['iceberg']==1]
test

,ORDERNO,VOLUME,iceberg,time_identified
ORDERNO,,,,
86829,86829,-400,1,100142388
107058,107058,-10500,1,100237033
113992,113992,-7500,1,100238747
120434,120434,-7000,1,100820867
147575,147575,-3400,1,100409587


In [120]:
iceberg[]

,ORDERNO,VOLUME,TIME
0,9367,300,100000117
1,5591,300,100000117
2,9405,900,100000322
3,5591,900,100000322
4,9424,2000,100000343
...,...,...,...
2059,303960,2500,100959790
2060,303959,1000,100959793
2061,303965,2700,100959901
2062,303989,2300,100959926


In [126]:
#ice_detected - ДФ, содержащий все вскрытые заявки типа "айсберг" за выбранный период времени
#Примечание: значение VOLUME отражает объем, на котором айсберг был вскрыт. 
#В ходе дальнейших торгов VOLUME мог и дальше уходить в отрицательную зону
ice_detected = place[place['iceberg']==1]
ice_detected.index.names = ['Index']
#количество вскрытых айсбергов за выбранный период времени
ice_numb = ice_detected['iceberg'].sum()

In [144]:
#добавление времени выставления заявок, содержащих айсберги
place = data[data['ACTION']=='1']
place_chd = place[['ORDERNO', 'BUYSELL','PRICE', 'TIME']]
output = pd.merge(ice_detected, place_chd, on = 'ORDERNO', how = 'left')
output['delta'] = output['time_identified'] - output['TIME']
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta
0,86829,-400,1,100142388,S,65.64,100141619,769
1,107058,-10500,1,100237033,S,65.69,100222955,14078
2,113992,-7500,1,100238747,S,65.69,100237043,1704
3,120434,-7000,1,100820867,S,65.69,100253305,567562
4,147575,-3400,1,100409587,B,65.51,100359198,50389


In [128]:
     for p in range (0, len(output)):
        #переводим в секунды с округлением миллисекунд по правилам математики
        if output.loc[p, 'time_identified'] % 1000 >= 500:
            output.loc[p, 'time_identified'] = output.loc[p, 'time_identified'] // 1000 + 1
        else:
            output.loc[p, 'time_identified'] = output.loc[p, 'time_identified'] // 1000
        #выделяем количество часов
        hours_1 = output.loc[p, 'time_identified']//10000
        #выделяем количество минут
        minutes_1 = output.loc[p, 'time_identified']//100 - hours_1*100
        #выделяем количество секунд
        seconds_1 = output.loc[p, 'time_identified'] - hours_1*10000 - minutes_1*100
        
        if output.loc[p, 'TIME'] % 1000 >= 500:
            output.loc[p, 'TIME'] = output.loc[p, 'TIME'] // 1000 + 1
        else:
            output.loc[p, 'TIME'] = output.loc[p, 'TIME'] // 1000
        hours_2 = output.loc[p, 'TIME']//10000
        minutes_2 = output.loc[p, 'TIME']//100 - hours_2*100
        seconds_2 = output.loc[p, 'TIME'] - hours_2*10000 - minutes_2*100
        
        if minutes_1 >= minutes_2:
            if seconds_1 >= seconds_2:
                output.loc[p, 'delta'] = (hours_1 - hours_2)*3600 + (minutes_1 - minutes_2)*60 + (seconds_1 - seconds_2)
            else:
                output.loc[p, 'delta'] = (hours_1 - hours_2)*3600 + (minutes_1 - 1 - minutes_2)*60 + (seconds_1 + 60 - seconds_2)
        else:
            if seconds_1 >= seconds_2:
                output.loc[p, 'delta'] = (hours_1 - 1 - hours_2)*3600 + (minutes_1 + 60 - minutes_2)*60 + (seconds_1 - seconds_2)
            else:
                output.loc[p, 'delta'] = (hours_1 - 1 - hours_2)*3600 + (minutes_1 + 59 - minutes_2)*60 + (seconds_1 + 60 - seconds_2)

In [145]:
for p in range (0, len(output)):
    #переводим в секунды с округлением миллисекунд по правилам математики
    if output.loc[p, 'time_identified'] % 1000 >= 500:
        output.loc[p, 'time_identified'] = output.loc[p, 'time_identified'] // 1000 + 1
    else:
        output.loc[p, 'time_identified'] = output.loc[p, 'time_identified'] // 1000
    hours_1 = output.loc[p, 'time_identified']//10000
    minutes_1 = output.loc[p, 'time_identified']//100 - hours_1*100
    print(minutes_1)

1
2
2
8
4


In [18]:
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta,Bid/Ask,numb_of_steps
0,86829,-400,1,100142,S,65.64,100142,0,0.05,5.0
1,107058,-10500,1,100237,S,65.69,100223,14,0.01,1.0
2,113992,-7500,1,100239,S,65.69,100237,2,0.03,3.0
3,120434,-7000,1,100821,S,65.69,100253,328,0.00,0.0
4,147575,-3400,1,100410,B,65.51,100359,11,0.07,7.0
...,...,...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12,0.00,0.0
164,6734093,-200,1,174831,B,65.93,174831,0,-0.01,-1.0
165,6805069,-1300,1,175549,B,66.05,175549,0,-0.04,-4.0
166,6805439,-3000,1,175552,B,66.05,175552,0,0.00,0.0


In [20]:
place_from_ask_bid(output, price_step)
output

,ORDERNO,VOLUME,iceberg,time_identified,BUYSELL,PRICE,TIME,delta,Bid/Ask,numb_of_steps
0,86829,-400,1,100142,S,65.64,100142,0,0.05,5.0
1,107058,-10500,1,100237,S,65.69,100223,14,0.01,1.0
2,113992,-7500,1,100239,S,65.69,100237,2,0.03,3.0
3,120434,-7000,1,100821,S,65.69,100253,328,0.00,0.0
4,147575,-3400,1,100410,B,65.51,100359,11,0.07,7.0
...,...,...,...,...,...,...,...,...,...,...
163,6731494,-3400,1,174831,S,65.98,174819,12,0.00,0.0
164,6734093,-200,1,174831,B,65.93,174831,0,-0.01,-1.0
165,6805069,-1300,1,175549,B,66.05,175549,0,-0.04,-4.0
166,6805439,-3000,1,175552,B,66.05,175552,0,0.00,0.0


In [48]:
from datetime import datetime
d = '29.11.2020'
def ReverseToDatetime(date, ordertime):
    hh24 = ordertime//10000000 # выделяем разряды, соответствующие часам
    mi = ordertime//100000%100 # выделяем разряды, соответствующие минутам
    ss = ordertime//1000%100 # выделяем разряды, соответствующие секундам
    ff3 = ordertime%1000 # выделяем разряды, соответствующие микросекундам
    reversed_time = datetime.strptime(date + ' ' + str(hh24) + ':' + str(mi) + ':' + str(ss) + ',' + str(ff3), '%d.%m.%Y %H:%M:%S,%f') #переводим строку в формат времени для python
    return(reversed_time)

In [8]:
ReverseToDatetime(date,time_upper)

datetime.datetime(2020, 4, 29, 18, 0)

In [20]:
test = ReverseToDatetime(date,data_from_postgres['TIME'][5550])
print(test)

2020-04-29 10:10:56.840000


In [22]:
from datetime import datetime, date, time

In [62]:
#если время прочитано как число
t = time(time_upper//10000000,time_upper//100000%100,time_upper//1000%100,time_upper%1000) #выделяем из числа чч:мм:сс,мс
t
#если время прочитано как строка
#t = time(int(t[0:2]),int(t[2:4]),int(t[4:6]),int(t[6:12]))

datetime.time(18, 0)

In [67]:
d = '29.11.2020'
d = date(int(d[6:10]),int(d[3:5]),int(d[0:2])) #парсим строку с введенной датой на гггг.мм.дд
d

datetime.date(2020, 11, 29)

In [64]:
datetime.combine(d, t)

datetime.datetime(2020, 11, 29, 18, 0)

In [99]:
def ReverseToDatetime(t):
    t = str(t)
    t = time(int(t[0:2]),int(t[2:4]),int(t[4:6]),int(t[6:9])) #выделяем из числа чч:мм:сс,мс
    d = '29.11.2020'
    d = date(int(d[6:10]),int(d[3:5]),int(d[0:2])) #парсим строку с введенной датой на гггг.мм.дд
    dt = datetime.combine(d, t)
    return(dt)

In [74]:
d = '29.11.2020'
time_upper = 112233123456
dt = ReverseToDatetime(d, time_upper)
dt

datetime.datetime(2020, 11, 29, 11, 22, 33, 123456)

In [85]:
ReverseToDatetime(d,data_from_postgres['TIME'][0])

datetime.datetime(2020, 11, 29, 10, 0)

In [86]:
data_from_postgres['TIME1'] = ReverseToDatetime(d,data_from_postgres['TIME'])

ValueError: invalid literal for int() with base 10: '  '

In [92]:
data_from_postgres['TIME1'] = 0

In [93]:
data_from_postgres

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TIME1
0,2,ALRS,S,100000000,2,1,67.75,300,NaN,NaN,0
1,87,ALRS,S,100000000,87,1,66.36,100,NaN,NaN,0
2,313,ALRS,S,100000000,313,1,66.28,2000,NaN,NaN,0
3,406,ALRS,B,100000000,406,1,65.00,100,NaN,NaN,0
4,486,ALRS,S,100000000,486,1,69.63,900,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
122288,14075136,ALRS,S,175940800,6849776,0,66.23,200,NaN,NaN,0
122289,14075145,ALRS,S,175940812,6850696,1,66.27,200,NaN,NaN,0
122290,14076935,ALRS,B,175947977,6851573,1,66.11,700,NaN,NaN,0
122291,14078327,ALRS,B,175953818,6852259,1,66.15,400,NaN,NaN,0


In [96]:
for i in range (0, len(data_from_postgres)):
    data_from_postgres.loc[i, 'TIME1'] = ReverseToDatetime(d,data_from_postgres.loc[i, 'TIME1'])

ValueError: invalid literal for int() with base 10: ''

In [101]:
data_from_postgres['TIME'] = data_from_postgres['TIME'].apply(ReverseToDatetime)

ValueError: invalid literal for int() with base 10: '1-2'

In [105]:
data_from_postgres

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,2,ALRS,S,2020-11-29 10:00:00.000000,2,1,67.75,300,NaN,NaN
1,87,ALRS,S,2020-11-29 10:00:00.000000,87,1,66.36,100,NaN,NaN
2,313,ALRS,S,2020-11-29 10:00:00.000000,313,1,66.28,2000,NaN,NaN
3,406,ALRS,B,2020-11-29 10:00:00.000000,406,1,65.00,100,NaN,NaN
4,486,ALRS,S,2020-11-29 10:00:00.000000,486,1,69.63,900,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
122288,14075136,ALRS,S,2020-11-29 17:59:40.000800,6849776,0,66.23,200,NaN,NaN
122289,14075145,ALRS,S,2020-11-29 17:59:40.000812,6850696,1,66.27,200,NaN,NaN
122290,14076935,ALRS,B,2020-11-29 17:59:47.000977,6851573,1,66.11,700,NaN,NaN
122291,14078327,ALRS,B,2020-11-29 17:59:53.000818,6852259,1,66.15,400,NaN,NaN


In [104]:
del data_from_postgres['TIME1']